## Урок 2. Домашнее задание

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

import os, sys
sys.path.insert(1, os.getcwd() + '/webinar_2')
from metrics import precision_at_k, recall_at_k

In [2]:
data = pd.read_csv('webinar_2/data/retail_train.csv')
data.head()

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0


In [3]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

__Решение:__

In [4]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


Для начала добавим случайную рекомендацию и рекомендацию на основе популярности, представленные на лекции.

In [5]:
def random_recommendation(items, n=5):
    """Случайные рекоммендации"""
    
    items = np.array(items)
    recs = np.random.choice(items, size=n, replace=False)
    
    return recs.tolist()

In [6]:
%%time

items = data_train.item_id.unique()

result['random_recommendation'] = result['user_id'].apply(lambda x: random_recommendation(items, n=5))

result.head(2)

CPU times: user 3.39 s, sys: 212 ms, total: 3.6 s
Wall time: 3.69 s


,user_id,actual,random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[915170, 9836679, 142497, 15629552, 938991]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[13008113, 1574905, 975770, 6979618, 13513000]"


In [7]:
def popularity_recommendation(data, n=5):
    """Топ-n популярных товаров"""
    
    popular = data.groupby('item_id')['sales_value'].sum().reset_index()
    popular.sort_values('sales_value', ascending=False, inplace=True)
    
    recs = popular.head(n).item_id
    
    return recs.tolist()

In [8]:
%%time

# Можно так делать, так как рекомендация не зависит от юзера
popular_recs = popularity_recommendation(data_train, n=5)

result['popular_recommendation'] = result['user_id'].apply(lambda x: popular_recs)
result.head(2)

CPU times: user 112 ms, sys: 16.5 ms, total: 128 ms
Wall time: 135 ms


,user_id,actual,random_recommendation,popular_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[915170, 9836679, 142497, 15629552, 938991]","[6534178, 6533889, 1029743, 6534166, 1082185]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[13008113, 1574905, 975770, 6979618, 13513000]","[6534178, 6533889, 1029743, 6534166, 1082185]"


Теперь добавим случайную взвешенную рекомендацию.

In [9]:
def weighted_random_recommendation(items_weights, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    
    recs = np.random.choice(items_weights['item_id'], size=n, replace=False, p=items_weights['weight'])
    return recs.tolist()

In [10]:
### Получаем датафрейм item_id - total sales_value
df = data.groupby('item_id')['sales_value'].sum().reset_index()

### Заменяем нулевые значения, чтобы можно было посчитать логарифм
df['sales_value'] = np.where(df['sales_value'] == 0, 1e-15, df['sales_value'])
df.head()

,item_id,sales_value
0,25671,20.94
1,26081,0.99
2,26093,1.59
3,26190,1.54
4,26355,1.98


Я не придумал как наиболее корректно перейти от логарифма продаж к весам, которые должны быть во-первых неотрицательными, а во-вторых их сумма должна быть равна единице. Поэтому попробовал разные варианты.

1. min-max нормирование логарифма продаж

In [11]:
%%time

df['log_sales_value'] = np.log(df['sales_value'])
df['log_sales_value'] = (df['log_sales_value'] - min(df['log_sales_value']))\
                        /(max(df['log_sales_value']) - min(df['log_sales_value']))
df['weight'] = df['log_sales_value']/sum(df['log_sales_value'])
df.head()

CPU times: user 35.5 ms, sys: 2.5 ms, total: 38 ms
Wall time: 39 ms


,item_id,sales_value,log_sales_value,weight
0,25671,20.94,0.790111,0.000011
1,26081,0.99,0.726152,0.000010
2,26093,1.59,0.736082,0.000011
3,26190,1.54,0.735412,0.000011
4,26355,1.98,0.740679,0.000011


In [12]:
result['weighted_random_recommendation_1'] = result['user_id'].\
                                            apply(lambda x: weighted_random_recommendation(df, n=5))
result.head()

,user_id,actual,random_recommendation,popular_recommendation,weighted_random_recommendation_1
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[915170, 9836679, 142497, 15629552, 938991]","[6534178, 6533889, 1029743, 6534166, 1082185]","[12811200, 8069177, 9439495, 6602297, 948626]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[13008113, 1574905, 975770, 6979618, 13513000]","[6534178, 6533889, 1029743, 6534166, 1082185]","[970476, 5592570, 12183264, 832858, 851683]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[8293633, 863699, 1119785, 12394806, 15742009]","[6534178, 6533889, 1029743, 6534166, 1082185]","[5587555, 1778473, 12673365, 924266, 1360960]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[42102, 10255189, 8293139, 12781482, 15741196]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1111825, 911493, 12188388, 13871483, 9553251]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[2527108, 980951, 1007169, 1587420, 974524]","[6534178, 6533889, 1029743, 6534166, 1082185]","[9213586, 12757123, 1407216, 2356722, 15925548]"


2. min-max нормирование взвешенного логарифма продаж

In [13]:
%%time

df['log_sales_value'] = np.log(df['sales_value'])
df['log_sales_value'] = df['log_sales_value'] / np.log(sum(df['sales_value']))
df['log_sales_value'] = (df['log_sales_value'] - min(df['log_sales_value']))\
                        /(max(df['log_sales_value']) - min(df['log_sales_value']))
df['weight'] = df['log_sales_value']/sum(df['log_sales_value'])
df.head()

CPU times: user 43.6 ms, sys: 2.93 ms, total: 46.6 ms
Wall time: 52 ms


,item_id,sales_value,log_sales_value,weight
0,25671,20.94,0.790111,0.000011
1,26081,0.99,0.726152,0.000010
2,26093,1.59,0.736082,0.000011
3,26190,1.54,0.735412,0.000011
4,26355,1.98,0.740679,0.000011


In [14]:
result['weighted_random_recommendation_2'] = result['user_id'].\
                                            apply(lambda x: weighted_random_recommendation(df, n=5))
result.head()

,user_id,actual,random_recommendation,popular_recommendation,weighted_random_recommendation_1,weighted_random_recommendation_2
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[915170, 9836679, 142497, 15629552, 938991]","[6534178, 6533889, 1029743, 6534166, 1082185]","[12811200, 8069177, 9439495, 6602297, 948626]","[7166462, 840322, 1025295, 15630064, 838867]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[13008113, 1574905, 975770, 6979618, 13513000]","[6534178, 6533889, 1029743, 6534166, 1082185]","[970476, 5592570, 12183264, 832858, 851683]","[999217, 7410305, 10462531, 16733779, 948381]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[8293633, 863699, 1119785, 12394806, 15742009]","[6534178, 6533889, 1029743, 6534166, 1082185]","[5587555, 1778473, 12673365, 924266, 1360960]","[12263267, 15972484, 954086, 1313235, 13381263]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[42102, 10255189, 8293139, 12781482, 15741196]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1111825, 911493, 12188388, 13871483, 9553251]","[14025422, 5591784, 10457247, 7466907, 1920561]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[2527108, 980951, 1007169, 1587420, 974524]","[6534178, 6533889, 1029743, 6534166, 1082185]","[9213586, 12757123, 1407216, 2356722, 15925548]","[6919320, 1642893, 15717202, 6513785, 1082554]"


3. Зануление весов для товаров, у которых логарифм продаж отрицательный. Так как логарифм отрицательный только для маленьких объемов продаж, то логично, что мы хотим рекомендовать такие товары с меньшей вероятностью

In [15]:
%%time

df['log_sales_value'] = np.log(df['sales_value'])
df.loc[df['log_sales_value'] < 0, 'log_sales_value'] = 0
df['weight'] = df['log_sales_value']/sum(df['log_sales_value'])
df.head()

CPU times: user 14.4 ms, sys: 2.16 ms, total: 16.5 ms
Wall time: 16.9 ms


,item_id,sales_value,log_sales_value,weight
0,25671,20.94,3.041661,0.000013
1,26081,0.99,0.000000,0.000000
2,26093,1.59,0.463734,0.000002
3,26190,1.54,0.431782,0.000002
4,26355,1.98,0.683097,0.000003


In [16]:
result['weighted_random_recommendation_3'] = result['user_id'].\
                                            apply(lambda x: weighted_random_recommendation(df, n=5))
result.head()

,user_id,actual,random_recommendation,popular_recommendation,weighted_random_recommendation_1,weighted_random_recommendation_2,weighted_random_recommendation_3
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[915170, 9836679, 142497, 15629552, 938991]","[6534178, 6533889, 1029743, 6534166, 1082185]","[12811200, 8069177, 9439495, 6602297, 948626]","[7166462, 840322, 1025295, 15630064, 838867]","[1305358, 1077085, 15831250, 1074693, 1014238]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[13008113, 1574905, 975770, 6979618, 13513000]","[6534178, 6533889, 1029743, 6534166, 1082185]","[970476, 5592570, 12183264, 832858, 851683]","[999217, 7410305, 10462531, 16733779, 948381]","[15452869, 937075, 1076448, 915651, 1017572]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[8293633, 863699, 1119785, 12394806, 15742009]","[6534178, 6533889, 1029743, 6534166, 1082185]","[5587555, 1778473, 12673365, 924266, 1360960]","[12263267, 15972484, 954086, 1313235, 13381263]","[823850, 1056641, 850501, 828085, 10457178]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[42102, 10255189, 8293139, 12781482, 15741196]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1111825, 911493, 12188388, 13871483, 9553251]","[14025422, 5591784, 10457247, 7466907, 1920561]","[6034283, 1041196, 12427035, 977873, 999183]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[2527108, 980951, 1007169, 1587420, 974524]","[6534178, 6533889, 1029743, 6534166, 1082185]","[9213586, 12757123, 1407216, 2356722, 15925548]","[6919320, 1642893, 15717202, 6513785, 1082554]","[6391203, 6544219, 1269270, 5995819, 15801279]"


4. Взвешенный объем продаж, без логарифмов

In [17]:
%%time

df = data.groupby('item_id')['sales_value'].sum().reset_index()
df['w_sales_value'] = df['sales_value']/ np.sum(df['sales_value'])
df['weight'] = df['w_sales_value']/sum(df['w_sales_value'])
df.head()

CPU times: user 105 ms, sys: 9.34 ms, total: 114 ms
Wall time: 121 ms


,item_id,sales_value,w_sales_value,weight
0,25671,20.94,2.817523e-06,2.817523e-06
1,26081,0.99,1.332067e-07,1.332067e-07
2,26093,1.59,2.139380e-07,2.139380e-07
3,26190,1.54,2.072103e-07,2.072103e-07
4,26355,1.98,2.664133e-07,2.664133e-07


In [18]:
result['weighted_random_recommendation_4'] = result['user_id'].\
                                            apply(lambda x: weighted_random_recommendation(df, n=5))
result.head()

,user_id,actual,random_recommendation,popular_recommendation,weighted_random_recommendation_1,weighted_random_recommendation_2,weighted_random_recommendation_3,weighted_random_recommendation_4
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[915170, 9836679, 142497, 15629552, 938991]","[6534178, 6533889, 1029743, 6534166, 1082185]","[12811200, 8069177, 9439495, 6602297, 948626]","[7166462, 840322, 1025295, 15630064, 838867]","[1305358, 1077085, 15831250, 1074693, 1014238]","[6533765, 6534178, 944137, 1116601, 13777454]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[13008113, 1574905, 975770, 6979618, 13513000]","[6534178, 6533889, 1029743, 6534166, 1082185]","[970476, 5592570, 12183264, 832858, 851683]","[999217, 7410305, 10462531, 16733779, 948381]","[15452869, 937075, 1076448, 915651, 1017572]","[1123496, 975649, 1041758, 10149640, 1079913]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[8293633, 863699, 1119785, 12394806, 15742009]","[6534178, 6533889, 1029743, 6534166, 1082185]","[5587555, 1778473, 12673365, 924266, 1360960]","[12263267, 15972484, 954086, 1313235, 13381263]","[823850, 1056641, 850501, 828085, 10457178]","[5569230, 6534178, 993638, 1076703, 988277]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[42102, 10255189, 8293139, 12781482, 15741196]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1111825, 911493, 12188388, 13871483, 9553251]","[14025422, 5591784, 10457247, 7466907, 1920561]","[6034283, 1041196, 12427035, 977873, 999183]","[1082176, 12781242, 5569601, 1008547, 6533889]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[2527108, 980951, 1007169, 1587420, 974524]","[6534178, 6533889, 1029743, 6534166, 1082185]","[9213586, 12757123, 1407216, 2356722, 15925548]","[6919320, 1642893, 15717202, 6513785, 1082554]","[6391203, 6544219, 1269270, 5995819, 15801279]","[1053690, 864214, 845208, 882830, 960253]"


Сравним метрики, рассчитанные для этих вариантов:

In [19]:
for name_col in result.columns[1:]:
    print(f"{round(result.apply(lambda row: precision_at_k(row[name_col], row['actual']), axis=1).mean(),4)}:{name_col}")

1.0:actual
0.0002:random_recommendation
0.1552:popular_recommendation
0.0008:weighted_random_recommendation_1
0.0005:weighted_random_recommendation_2
0.002:weighted_random_recommendation_3
0.0244:weighted_random_recommendation_4


Как видим значения метрик для рекомендаций, сделанных путем случайного выбора товаров с весами на основе логарифмов продаж, практически не отличаются от абсолютно случайного выбора товаров. Скорее всего это связано с большим количеством товаров в выборке и с тем, что логарифмируя мы несколько сглаживаем существующий разброс в продажах. <br>
Метрика для рекомендации, когда мы случайно выбираем товары, но используем в качестве весов просто взвешенный объем продаж, без логарифмов, получается лучше, но все равно она сильно уступает рекомендации на основе популярности, представленной на лекции.

## Задание 2. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая случаный на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.

__Решение:__ <br>
1. Для начала посчитаем случайную рекомендацию, которая делается на основе топ-5000 товаров.

In [20]:
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

popularity.head()

,item_id,n_sold
0,25671,6
1,26081,1
2,26093,1
3,26190,1
4,26355,2


In [21]:
top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

In [22]:
result['random_recommendation_top5000'] = result['user_id'].apply(lambda x: random_recommendation(top_5000, n=5))

result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,weighted_random_recommendation_1,weighted_random_recommendation_2,weighted_random_recommendation_3,weighted_random_recommendation_4,random_recommendation_top5000
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[915170, 9836679, 142497, 15629552, 938991]","[6534178, 6533889, 1029743, 6534166, 1082185]","[12811200, 8069177, 9439495, 6602297, 948626]","[7166462, 840322, 1025295, 15630064, 838867]","[1305358, 1077085, 15831250, 1074693, 1014238]","[6533765, 6534178, 944137, 1116601, 13777454]","[950384, 986912, 9705473, 9553042, 857276]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[13008113, 1574905, 975770, 6979618, 13513000]","[6534178, 6533889, 1029743, 6534166, 1082185]","[970476, 5592570, 12183264, 832858, 851683]","[999217, 7410305, 10462531, 16733779, 948381]","[15452869, 937075, 1076448, 915651, 1017572]","[1123496, 975649, 1041758, 10149640, 1079913]","[907014, 894886, 1083840, 1033857, 5592094]"


In [23]:
%%time

df = data.groupby('item_id')['sales_value'].sum().reset_index()
df_top5000 = df[df['item_id'].isin(top_5000)].copy()
df_top5000['sales_value'] = np.where(df_top5000['sales_value'] == 0, 1e-15, df_top5000['sales_value'])
df_top5000['log_sales_value'] = np.log(df_top5000['sales_value'])
df_top5000.loc[df_top5000['log_sales_value'] < 0, 'log_sales_value'] = 0
df_top5000['weight'] = df_top5000['log_sales_value']/sum(df_top5000['log_sales_value'])
df_top5000.head()

CPU times: user 101 ms, sys: 8.35 ms, total: 109 ms
Wall time: 115 ms


,item_id,sales_value,log_sales_value,weight
2733,202291,81.59,4.401707,0.000146
3460,397896,2932.59,7.983641,0.000264
3631,420647,463.81,6.139475,0.000203
3939,480014,912.63,6.816331,0.000226
4272,545926,58.37,4.066802,0.000135


In [24]:
result['weighted_random_recommendation_3_top5000'] = result['user_id'].\
                                            apply(lambda x: weighted_random_recommendation(df_top5000, n=5))
result.head()

,user_id,actual,random_recommendation,popular_recommendation,weighted_random_recommendation_1,weighted_random_recommendation_2,weighted_random_recommendation_3,weighted_random_recommendation_4,random_recommendation_top5000,weighted_random_recommendation_3_top5000
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[915170, 9836679, 142497, 15629552, 938991]","[6534178, 6533889, 1029743, 6534166, 1082185]","[12811200, 8069177, 9439495, 6602297, 948626]","[7166462, 840322, 1025295, 15630064, 838867]","[1305358, 1077085, 15831250, 1074693, 1014238]","[6533765, 6534178, 944137, 1116601, 13777454]","[950384, 986912, 9705473, 9553042, 857276]","[918400, 888614, 1007210, 1019637, 6534480]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[13008113, 1574905, 975770, 6979618, 13513000]","[6534178, 6533889, 1029743, 6534166, 1082185]","[970476, 5592570, 12183264, 832858, 851683]","[999217, 7410305, 10462531, 16733779, 948381]","[15452869, 937075, 1076448, 915651, 1017572]","[1123496, 975649, 1041758, 10149640, 1079913]","[907014, 894886, 1083840, 1033857, 5592094]","[1023979, 917816, 890536, 981521, 10149640]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[8293633, 863699, 1119785, 12394806, 15742009]","[6534178, 6533889, 1029743, 6534166, 1082185]","[5587555, 1778473, 12673365, 924266, 1360960]","[12263267, 15972484, 954086, 1313235, 13381263]","[823850, 1056641, 850501, 828085, 10457178]","[5569230, 6534178, 993638, 1076703, 988277]","[985119, 994868, 12384365, 7410013, 1012873]","[892004, 878715, 5568758, 957232, 938512]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[42102, 10255189, 8293139, 12781482, 15741196]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1111825, 911493, 12188388, 13871483, 9553251]","[14025422, 5591784, 10457247, 7466907, 1920561]","[6034283, 1041196, 12427035, 977873, 999183]","[1082176, 12781242, 5569601, 1008547, 6533889]","[5569098, 1074697, 5591038, 1079048, 1061394]","[1096343, 9487606, 994262, 958252, 6534178]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[2527108, 980951, 1007169, 1587420, 974524]","[6534178, 6533889, 1029743, 6534166, 1082185]","[9213586, 12757123, 1407216, 2356722, 15925548]","[6919320, 1642893, 15717202, 6513785, 1082554]","[6391203, 6544219, 1269270, 5995819, 15801279]","[1053690, 864214, 845208, 882830, 960253]","[1006878, 888404, 1022003, 12949855, 1056005]","[901060, 6391532, 959409, 837865, 968992]"


In [25]:
for name_col in result.columns[1:]:
    print(f"{round(result.apply(lambda row: precision_at_k(row[name_col], row['actual']), axis=1).mean(),4)}:{name_col}")

1.0:actual
0.0002:random_recommendation
0.1552:popular_recommendation
0.0008:weighted_random_recommendation_1
0.0005:weighted_random_recommendation_2
0.002:weighted_random_recommendation_3
0.0244:weighted_random_recommendation_4
0.0067:random_recommendation_top5000
0.0068:weighted_random_recommendation_3_top5000


Видно, что метрики для случайных рекомендаций, выдаваемых на основе отобранных 5000 наиболее популярных по количеству продаж товаров, лучше чем для тех, что осуществляются на общей массе товаров.

2. Посчитаем для разных значений числа соседей $K$ 

Для начала скопируем необходимые преобразования из лекции.

In [26]:
# Заведем фиктивный item_id (если юзер покупал товары из топ-5000, то он "купил" такой товар)
data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 6666

/usr/local/lib/python3.9/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [27]:
user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity',
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix[user_item_matrix > 0] = 1 # так как в итоге хотим предсказать 

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат sparse matrix
sparse_user_item = csr_matrix(user_item_matrix)

In [28]:
# создаем словари мапинга между id бизнеса к строчному id матрицы

userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [29]:
%%time

k = np.arange(1, 16)

for i in k:
    model = ItemItemRecommender(K=i, num_threads=4) # K - кол-во билжайших соседей


    model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
              show_progress=False)


    result[f'itemitem_{i}'] = result['user_id'].apply(lambda user_id: [
                                     id_to_itemid[rec[0]]  for rec in model.recommend(userid=userid_to_id[user_id], 
                                                user_items=sparse_user_item,   # на вход user-item matrix
                                                N=5, 
                                                filter_already_liked_items=False, 
                                                filter_items=[itemid_to_id[6666]], 
                                                recalculate_user=True)])

CPU times: user 23.1 s, sys: 307 ms, total: 23.4 s
Wall time: 10.5 s


In [31]:
for name_col in result.columns[1:]:
    print(f"{round(result.apply(lambda row: precision_at_k(row[name_col], row['actual']), axis=1).mean(),4)}:{name_col}")

1.0:actual
0.0002:random_recommendation
0.1552:popular_recommendation
0.0008:weighted_random_recommendation_1
0.0005:weighted_random_recommendation_2
0.002:weighted_random_recommendation_3
0.0244:weighted_random_recommendation_4
0.0067:random_recommendation_top5000
0.0068:weighted_random_recommendation_3_top5000
0.2199:itemitem_1
0.2193:itemitem_2
0.219:itemitem_3
0.1673:itemitem_4
0.151:itemitem_5
0.1555:itemitem_6
0.16:itemitem_7
0.1648:itemitem_8
0.1642:itemitem_9
0.1661:itemitem_10
0.1661:itemitem_11
0.1664:itemitem_12
0.1663:itemitem_13
0.1665:itemitem_14
0.1659:itemitem_15


Максимальное значение метрики достигается в случае ItemItem рекомендации, которая строится для одного ближайшего соседа ($K = 1$). Однако метрики для $K = 2$ и $K = 3$ тоже очень близки. Для больших значений $K$ метрика стабилизируется плюс-минус на одном уровне.

Посмотрим, как повлияет на метрики, если мы не будем применять бинарный подход в отношении количества продаж.

In [32]:
user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity',
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат sparse matrix
sparse_user_item = csr_matrix(user_item_matrix)

In [33]:
model = ItemItemRecommender(K=2, num_threads=4) # K - кол-во билжайших соседей


model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
              show_progress=False)


result[f'itemitem_non_binary_k2'] = result['user_id'].apply(lambda user_id: [
                                     id_to_itemid[rec[0]]  for rec in model.recommend(userid=userid_to_id[user_id], 
                                                user_items=sparse_user_item,   # на вход user-item matrix
                                                N=5, 
                                                filter_already_liked_items=False, 
                                                filter_items=[itemid_to_id[6666]], 
                                                recalculate_user=True)])

In [34]:
for name_col in result.columns[1:]:
    print(f"{round(result.apply(lambda row: precision_at_k(row[name_col], row['actual']), axis=1).mean(),4)}:{name_col}")

1.0:actual
0.0002:random_recommendation
0.1552:popular_recommendation
0.0008:weighted_random_recommendation_1
0.0005:weighted_random_recommendation_2
0.002:weighted_random_recommendation_3
0.0244:weighted_random_recommendation_4
0.0067:random_recommendation_top5000
0.0068:weighted_random_recommendation_3_top5000
0.2199:itemitem_1
0.2193:itemitem_2
0.219:itemitem_3
0.1673:itemitem_4
0.151:itemitem_5
0.1555:itemitem_6
0.16:itemitem_7
0.1648:itemitem_8
0.1642:itemitem_9
0.1661:itemitem_10
0.1661:itemitem_11
0.1664:itemitem_12
0.1663:itemitem_13
0.1665:itemitem_14
0.1659:itemitem_15
0.2994:itemitem_non_binary_k2


Видим, что метрика значительно выросла для построенной рекомендации.